In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import __version__ as sklearn_version
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, f_regression
import datetime
from sklearn.metrics import classification_report, confusion_matrix,recall_score , accuracy_score, precision_score, roc_auc_score

In [2]:
opera = pd.read_excel('/Users/angelique/Documents/GitHub/New_Opera_Company_Capstone/Excel and CSV Files/opera_4.xlsx')
opera.head()

,season,iso,city,composer,db,dd,nat,mf,work,worknat,...,performances_season_by_city,perf_per_1k_ppl_city_pop,opera_by_composer,performances_season_by_country_total,performances_season_by_city_total,perf_total_per_1k_city_pop,perf_total_per_10k_co_pop,Season Year,country_change_from_previous_season,city_change_from_previous_season
0,1213,al,Tirana,Lortzing,1801,1851,de,m,Ali Pascha von Janina,de,...,9,0.021506,Ali Pascha von Janina by Lor,110,110,0.262847,0.385301,2013-01-01,NaN,NaN
1,1213,al,Tirana,Mozart,1756,1791,at,m,Don Giovanni,it,...,9,0.021506,Don Giovanni by Moz,110,110,0.262847,0.385301,2013-01-01,NaN,NaN
2,1213,al,Tirana,Puccini,1858,1924,it,m,Tosca,it,...,9,0.021506,Tosca by Puc,110,110,0.262847,0.385301,2013-01-01,NaN,NaN
3,1213,am,Yerevan,Spendiaryan,1871,1928,am,m,Almast,am,...,5,0.004573,Almast by Spe,111,111,0.101510,0.371147,2013-01-01,NaN,NaN
4,1213,am,Yerevan,Tigranian,1879,1950,am,m,Anoush,am,...,5,0.004573,Anoush by Tig,111,111,0.101510,0.371147,2013-01-01,NaN,NaN


In [4]:
opera.columns

Index(['season', 'iso', 'city', 'composer', 'db', 'dd', 'nat', 'mf', 'work',
       'worknat', 'type', 'start date', 'performances', 'Country Name',
       'city population', 'country population', 'continent', 'sub-region',
       'performances_season_by_country', 'perf_per_10k_ppl_co_pop',
       'performances_season_by_city', 'perf_per_1k_ppl_city_pop',
       'opera_by_composer', 'performances_season_by_country_total',
       'performances_season_by_city_total', 'perf_total_per_1k_city_pop',
       'perf_total_per_10k_co_pop', 'Season Year',
       'country_change_from_previous_season',
       'city_change_from_previous_season'],
      dtype='object')

In [5]:
opera.dtypes

season                                           int64
iso                                             object
city                                            object
composer                                        object
db                                              object
dd                                              object
nat                                             object
mf                                              object
work                                            object
worknat                                         object
type                                            object
start date                              datetime64[ns]
performances                                     int64
Country Name                                    object
city population                                  int64
country population                             float64
continent                                       object
sub-region                                      object
performanc

In [6]:
opera['Year'] = opera['start date'].dt.year
opera['Month'] = opera['start date'].dt.month
opera['Weekday'] = opera['start date'].dt.weekday
opera['Week_of_Year'] = opera['start date'].dt.isocalendar().week
opera['Days_Since_Start'] = (opera['start date'] - opera['start date'].min()).dt.days
opera.head()

,season,iso,city,composer,db,dd,nat,mf,work,worknat,...,perf_total_per_1k_city_pop,perf_total_per_10k_co_pop,Season Year,country_change_from_previous_season,city_change_from_previous_season,Year,Month,Weekday,Week_of_Year,Days_Since_Start
0,1213,al,Tirana,Lortzing,1801,1851,de,m,Ali Pascha von Janina,de,...,0.262847,0.385301,2013-01-01,NaN,NaN,2013,3,5,12,273
1,1213,al,Tirana,Mozart,1756,1791,at,m,Don Giovanni,it,...,0.262847,0.385301,2013-01-01,NaN,NaN,2013,5,5,20,329
2,1213,al,Tirana,Puccini,1858,1924,it,m,Tosca,it,...,0.262847,0.385301,2013-01-01,NaN,NaN,2013,2,2,7,235
3,1213,am,Yerevan,Spendiaryan,1871,1928,am,m,Almast,am,...,0.101510,0.371147,2013-01-01,NaN,NaN,2013,7,3,28,383
4,1213,am,Yerevan,Tigranian,1879,1950,am,m,Anoush,am,...,0.101510,0.371147,2013-01-01,NaN,NaN,2013,5,5,19,322


I want to drop any columns that don't seem to be a factor or useful for any predictions

In [8]:
opera.drop(columns=['composer', 'db', 'dd', 'nat', 'mf', 'worknat', 'type'], inplace=True)
opera.columns

Index(['season', 'iso', 'city', 'work', 'start date', 'performances',
       'Country Name', 'city population', 'country population', 'continent',
       'sub-region', 'performances_season_by_country',
       'perf_per_10k_ppl_co_pop', 'performances_season_by_city',
       'perf_per_1k_ppl_city_pop', 'opera_by_composer',
       'performances_season_by_country_total',
       'performances_season_by_city_total', 'perf_total_per_1k_city_pop',
       'perf_total_per_10k_co_pop', 'Season Year',
       'country_change_from_previous_season',
       'city_change_from_previous_season', 'Year', 'Month', 'Weekday',
       'Week_of_Year', 'Days_Since_Start'],
      dtype='object')

I'm creating a function to assess the top values for cities and countries. This function will call up lists similar to the graphs shown in my EDA component that showed the top cities and top countries solely based on the delta change in performances from season to season as a total sum

In [177]:
def rank_top_values(df, group_column, value_column,top_n=25):
    """
    This function ranks values (cities, countries, or any other category) based on the sum of a numeric column
    and returns the top N rows with the highest sum of the specified value.

    Parameters:
    - df (pd.DataFrame): DataFrame containing the data to be ranked.
    - group_column (str): The column name to group by (e.g., 'city', 'country').
    - value_column (str): The numeric column name that will be summed and used for ranking (e.g., 'growth').
    - top_n (int): The number of top rows to return. Default is 25.

    Returns:
    - pd.DataFrame: DataFrame of the top N values with the highest summed numeric column.
    """
    # Group by the specified column and sum the values in the specified numeric column
    grouped = df.groupby([group_column, 'iso'], as_index=False)[value_column].sum()

    # Sort the grouped data by the value column in descending order (highest first)
    sorted_group = grouped.sort_values(by=value_column, ascending=False)

    # Get the top N rows based on the sorted values
    top_values = sorted_group.head(top_n)

    return top_values

In [179]:
top_cities = rank_top_values(opera, group_column='city', value_column='city_change_from_previous_season', top_n=25)
print(top_cities)

               city iso  city_change_from_previous_season
747          Moscow  ru                      30680.989959
742      Montevideo  uy                      20492.000000
1215     Washington  us                      20460.216359
1061  St Petersburg  ru                      10828.162888
51        Arlington  us                       9653.748405
758         München  de                       4028.783499
845           Paris  fr                       3921.544057
263        Columbus  us                       3520.789083
930        Richmond  us                       3098.877664
867    Philadelphia  us                       2892.026099
315         Dresden  de                       2395.427394
327      Dusseldorf  de                       1646.884232
891     Portland OR  us                       1609.833803
62          Atlanta  us                       1294.981580
371         Firenze  it                       1233.586430
295          Denver  us                       1192.761175
383       Fran

In [181]:
top_countries = rank_top_values(opera, group_column='Country Name', value_column='country_change_from_previous_season', top_n=25)
print(top_countries)

          Country Name iso  country_change_from_previous_season
33               Italy  it                        115768.351064
57  Russian Federation  ru                         74744.579768
71             Uruguay  uy                         20492.000000
64              Sweden  se                          5564.128067
54              Poland  pl                          4510.646139
49         Netherlands  nl                          3561.039044
31             Ireland  ie                          1329.715020
38              Latvia  lv                          1004.960434
36  Korea, Republic of  kr                           598.390353
1            Argentina  ar                           573.204941
72          Uzbekistan  uz                           489.450195
2              Armenia  am                           442.000000
13               China  cn                           366.968390
48            Mongolia  mn                           360.000000
3            Australia  au              

In [21]:
nan_count_per_column = opera.isna().sum()
print(nan_count_per_column)

season                                     0
iso                                        0
city                                       0
work                                       5
start date                                 0
performances                               0
Country Name                               0
city population                            0
country population                         0
continent                                  0
sub-region                                 0
performances_season_by_country             0
perf_per_10k_ppl_co_pop                    0
performances_season_by_city                0
perf_per_1k_ppl_city_pop                   0
opera_by_composer                          5
performances_season_by_country_total       0
performances_season_by_city_total          0
perf_total_per_1k_city_pop                 0
perf_total_per_10k_co_pop                  0
Season Year                                0
country_change_from_previous_season     6559
city_chang

In [23]:
opera.dropna(subset=['work'], inplace=True)

In [25]:
def predict_missing_values(df, target_column, feature_columns, group_column='iso'):
    # Step 1: Identify rows with missing values in the target column
    missing_data = df[df[target_column].isna()]
    
    # Step 2: Rows without missing values (training data)
    df_no_missing = df.dropna(subset=[target_column])
    
    # Separate features (X) and target (y)
    X_no_missing = df_no_missing[feature_columns]
    y_no_missing = df_no_missing[target_column]
    
    # Step 3: Train the Linear Regression Model for each group (e.g. 'iso')
    predicted_values = []
    missing_indices = []

    for group_value, group in df_no_missing.groupby(group_column):
        X_group = group[feature_columns]
        y_group = group[target_column]
        
        model = LinearRegression()
        model.fit(X_group, y_group)
        
        # Get the rows with missing target values in the current group
        missing_group = missing_data[missing_data[group_column] == group_value]
        
        # If there are missing values for this group, predict them
        if not missing_group.empty:
            X_missing = missing_group[feature_columns]
            predictions = model.predict(X_missing)
            
            # Append the predicted values and their indices
            predicted_values.extend(predictions)
            missing_indices.extend(missing_group.index)  # Track the indices for replacement

    # Step 4: Check if the predicted values match the missing rows
    if len(predicted_values) == len(missing_indices):
        # Replace the missing values with predictions
        df.loc[missing_indices, target_column] = predicted_values
    else:
        print(f"Length mismatch: {len(predicted_values)} predicted values, but {len(missing_indices)} missing rows.")

    return df

In [27]:
# Define the feature columns
feature_columns = ['city population', 'country population', 'performances_season_by_country',
                   'perf_per_10k_ppl_co_pop', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year']

# Predict missing values for 'country_change_from_previous_season'
opera = predict_missing_values(opera, 'country_change_from_previous_season', feature_columns)

# Predict missing values for 'city_change_from_previous_season'
opera = predict_missing_values(opera, 'city_change_from_previous_season', feature_columns)

# Check the result
print(opera.head())

   season iso     city                   work start date  performances  \
0    1213  al   Tirana  Ali Pascha von Janina 2013-03-23             4   
1    1213  al   Tirana           Don Giovanni 2013-05-18             3   
2    1213  al   Tirana                  Tosca 2013-02-13             2   
3    1213  am  Yerevan                 Almast 2013-07-11             1   
4    1213  am  Yerevan                 Anoush 2013-05-11             3   

  Country Name  city population  country population continent  ...  \
0      Albania           418495        2.854907e+06    Europe  ...   
1      Albania           418495        2.854907e+06    Europe  ...   
2      Albania           418495        2.854907e+06    Europe  ...   
3      Armenia          1093485        2.990731e+06      Asia  ...   
4      Armenia          1093485        2.990731e+06      Asia  ...   

  perf_total_per_1k_city_pop  perf_total_per_10k_co_pop  Season Year  \
0                   0.262847                   0.385301   2013

In [29]:
# One-hot encode 'Month', 'Weekday', and 'Day' 'Season Year' and 'opera_by_composer
opera = pd.get_dummies(opera, columns=['Month', 'Weekday', 'Season Year'], drop_first=False) 

# Check the result
opera.head()

,season,iso,city,work,start date,performances,Country Name,city population,country population,continent,...,Weekday_3,Weekday_4,Weekday_5,Weekday_6,Season Year_2013-01-01 00:00:00,Season Year_2014-01-01 00:00:00,Season Year_2015-01-01 00:00:00,Season Year_2016-01-01 00:00:00,Season Year_2017-01-01 00:00:00,Season Year_2018-01-01 00:00:00
0,1213,al,Tirana,Ali Pascha von Janina,2013-03-23,4,Albania,418495,2.854907e+06,Europe,...,False,False,True,False,True,False,False,False,False,False
1,1213,al,Tirana,Don Giovanni,2013-05-18,3,Albania,418495,2.854907e+06,Europe,...,False,False,True,False,True,False,False,False,False,False
2,1213,al,Tirana,Tosca,2013-02-13,2,Albania,418495,2.854907e+06,Europe,...,False,False,False,False,True,False,False,False,False,False
3,1213,am,Yerevan,Almast,2013-07-11,1,Armenia,1093485,2.990731e+06,Asia,...,True,False,False,False,True,False,False,False,False,False
4,1213,am,Yerevan,Anoush,2013-05-11,3,Armenia,1093485,2.990731e+06,Asia,...,False,False,True,False,True,False,False,False,False,False


In [31]:
# creation_source dummies are showing as boolean, switch to integer using the function below
def convert_booleans_to_integers(df):
    # Loop through columns and convert those with boolean dtype to integers
    for column in df.select_dtypes(include='bool').columns:
        df[column] = df[column].astype(int)
    return df

# Use new function
opera = convert_booleans_to_integers(opera)

# Check the first few rows to see the change
opera.head()

,season,iso,city,work,start date,performances,Country Name,city population,country population,continent,...,Weekday_3,Weekday_4,Weekday_5,Weekday_6,Season Year_2013-01-01 00:00:00,Season Year_2014-01-01 00:00:00,Season Year_2015-01-01 00:00:00,Season Year_2016-01-01 00:00:00,Season Year_2017-01-01 00:00:00,Season Year_2018-01-01 00:00:00
0,1213,al,Tirana,Ali Pascha von Janina,2013-03-23,4,Albania,418495,2.854907e+06,Europe,...,0,0,1,0,1,0,0,0,0,0
1,1213,al,Tirana,Don Giovanni,2013-05-18,3,Albania,418495,2.854907e+06,Europe,...,0,0,1,0,1,0,0,0,0,0
2,1213,al,Tirana,Tosca,2013-02-13,2,Albania,418495,2.854907e+06,Europe,...,0,0,0,0,1,0,0,0,0,0
3,1213,am,Yerevan,Almast,2013-07-11,1,Armenia,1093485,2.990731e+06,Asia,...,1,0,0,0,1,0,0,0,0,0
4,1213,am,Yerevan,Anoush,2013-05-11,3,Armenia,1093485,2.990731e+06,Asia,...,0,0,1,0,1,0,0,0,0,0


In [33]:
opera.dropna(subset=['country_change_from_previous_season'], inplace=True)
print(opera.isna().sum())

season                                  0
iso                                     0
city                                    0
work                                    0
start date                              0
performances                            0
Country Name                            0
city population                         0
country population                      0
continent                               0
sub-region                              0
performances_season_by_country          0
perf_per_10k_ppl_co_pop                 0
performances_season_by_city             0
perf_per_1k_ppl_city_pop                0
opera_by_composer                       0
performances_season_by_country_total    0
performances_season_by_city_total       0
perf_total_per_1k_city_pop              0
perf_total_per_10k_co_pop               0
country_change_from_previous_season     0
city_change_from_previous_season        0
Year                                    0
Week_of_Year                      

In [55]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


# Feature columns and target columns
feature_columns = ['city population', 'country population', 'performances_season_by_country',
                   'perf_per_10k_ppl_co_pop', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Week_of_Year',
                   'Days_Since_Start', 'Month_1', 'Month_2', 'Month_3', 'Month_4',
                   'Month_5', 'Month_6', 'Month_7', 'Month_8', 'Month_9', 'Month_10',
                   'Month_11', 'Month_12', 'Weekday_0', 'Weekday_1', 'Weekday_2',
                   'Weekday_3', 'Weekday_4', 'Weekday_5', 'Weekday_6',
                   'Season Year_2013-01-01 00:00:00', 'Season Year_2014-01-01 00:00:00',
                   'Season Year_2015-01-01 00:00:00', 'Season Year_2016-01-01 00:00:00',
                   'Season Year_2017-01-01 00:00:00', 'Season Year_2018-01-01 00:00:00']

# Group the data by 'sub-region' (or 'city' if applicable)
sub_region_groups = opera.groupby('sub-region')  # Group by 'sub-region'

# Initialize the linear regression model and scaler
model = LinearRegression()
scaler = StandardScaler()

# Function to evaluate linear regression
def evaluate_linear_regression(model, X, y_columns, sub_region_groups, scaler):
    """
    Trains and evaluates a linear regression model for each sub-region,
    then returns the average MAE, RMSE, and R² score for each target column,
    and also stores predictions for later sorting.
    """
    metrics = {}  # Dictionary to store metrics for each target column
    all_predictions = []  # List to store all predictions for sorting

    for target_column in y_columns:
        mae_scores, rmse_scores, r2_scores = [], [], []

        for country_or_city, group in sub_region_groups:
            if len(group) > 1:
                # Extract features and target
                X_group = group[X.columns]
                y_group = group[target_column]

                # Train-test split
                X_train, X_test, y_train, y_test = train_test_split(X_group, y_group, test_size=0.2, random_state=42)

                # Scale the data
                X_train_scaled = scaler.fit_transform(X_train)
                X_test_scaled = scaler.transform(X_test)

                # Train the model
                model.fit(X_train_scaled, y_train)

                # Make predictions
                predictions = model.predict(X_test_scaled)

                # Store predictions for sorting, including iso/sub-region
                for idx, pred in enumerate(predictions):
                    all_predictions.append({
                        'Name': group['Country Name'].iloc[idx] if target_column == 'country_change_from_previous_season' else group['city'].iloc[idx],  # or 'sub-region' if applicable
                        'iso': group['iso'].iloc[idx],  # add iso here
                        'Predicted Growth': pred
                    })

                # Compute evaluation metrics
                mae_scores.append(mean_absolute_error(y_test, predictions))
                rmse_scores.append(np.sqrt(mean_squared_error(y_test, predictions)))
                r2_scores.append(r2_score(y_test, predictions))

        # Compute average metrics for the current target column
        avg_metrics = {
            'MAE': np.mean(mae_scores) if mae_scores else 0,
            'RMSE': np.mean(rmse_scores) if rmse_scores else 0,
            'R2': np.mean(r2_scores) if r2_scores else 0
        }

        # Print each metric on a separate line
        print(f"Metrics for {target_column}:")
        print(f"  MAE: {avg_metrics['MAE']}")
        print(f"  RMSE: {avg_metrics['RMSE']}")
        print(f"  R2: {avg_metrics['R2']}")

        # Store metrics for this target column
        metrics[target_column] = avg_metrics

    # Sort predictions based on predicted growth (descending order)
    predictions_df = pd.DataFrame(all_predictions)
    predictions_df = predictions_df.sort_values(by='Predicted Growth', ascending=False)

    # Drop duplicates, keeping the row with the highest predicted growth for each country or city
    predictions_df_unique = predictions_df.drop_duplicates(subset='Name', keep='first')

    return metrics, predictions_df_unique

In [57]:
# List of target columns (for country prediction)
target_columns = ['country_change_from_previous_season']

# Call the function to evaluate the model for 'country_change_from_previous_season'
evaluation_results_country, predictions_df_country = evaluate_linear_regression(
    model, opera[feature_columns], target_columns, sub_region_groups, scaler)

# Get top 50 countries with highest predicted growth
top_countries_predicted = predictions_df_country.head(50)

# Print the top 10 predicted countries
print(top_countries_predicted.head(10))

Metrics for country_change_from_previous_season:
  MAE: 39.30978638759204
  RMSE: 52.203724902632906
  R2: -14.181765868919383
                    Name iso  Predicted Growth
1936              Brazil  br        575.052553
1974            Colombia  co        575.052553
1911           Argentina  ar        575.052553
1972               Chile  cl        514.740053
1995              Mexico  mx        394.552553
3282       United States  us        355.075120
3450              Canada  ca        351.075120
1988          Costa Rica  cr        343.552553
324             Bulgaria  bg        333.164941
1292  Russian Federation  ru        330.789941


In [59]:
# List of target columns (for city prediction)
target_columns = ['city_change_from_previous_season']

# Call the function to evaluate the model for 'country_change_from_previous_season'
evaluation_results_city, predictions_df_city = evaluate_linear_regression(
    model, opera[feature_columns], target_columns, sub_region_groups, scaler)

# Get top 50 countries with highest predicted growth
top_cities_predicted = predictions_df_city.head(50)

# Print the top 10 predicted countries
print(top_cities_predicted.head(10))

Metrics for city_change_from_previous_season:
  MAE: 24.56695758271886
  RMSE: 34.64250796283655
  R2: -17.393876634960527
                Name iso  Predicted Growth
1911    Buenos Aires  ar        574.664421
1955       São Paulo  br        574.664421
1974          Bogota  co        574.664421
1936          Manaus  br        574.664421
1972        Santiago  cl        508.351921
1923           Belem  br        412.976921
1995     Mexico City  mx        390.976921
1953  Rio de Janeiro  br        356.226921
1988        San José  cr        345.101921
1931  Belo Horizonte  br        299.726921


Linear Regression and grouping by sub-region for predictive analysis. I grouped the opera data by sub-regions before peforming the linear regression to see if it had accurate predictions.  The scores show that this model was not particularly strong for either of my target metrics.

Logistic Regression Models - let's see if we can get a better predictive model using Logistic regression without countries being grouped first and then try it with grouping countries. In order to run a Logistic regression model, I binned the country_change_from_previous_season by quartiles into categories of Low, Medium, High and Very High so there is a category that is being used as the dependent outcome.

In [63]:
# Using this function to evaluate performance of models
def evaluate_model_performance(model, X, y, sub_region_groups, scaler, bins, labels):
    """
    Trains and evaluates a model for each sub-region, then returns the average accuracy, precision, and recall.

    Parameters:
    - model: The machine learning model to train (e.g., RandomForestClassifier, LogisticRegression).
    - X: Feature DataFrame.
    - y: Target variable DataFrame.
    - country_groups: Grouped data by sub-region.
    - scaler: StandardScaler instance.
    - bins: Bins for categorizing target values.
    - labels: Labels corresponding to bins.

    Returns:
    - Dictionary with average accuracy, precision, and recall.
    """
    accuracies, precisions, recalls = [], [], []

    for sub_region, group in sub_region_groups:
        if len(group) > 1 and len(group['country_change_from_previous_season'].unique()) > 1:
            # Extract features and target
            X_group = group[X.columns]
            y_group = group[y.name]

            # Convert the target variable to categorical labels
            y_group_classified = pd.cut(y_group, bins=bins, labels=labels)

            # Train-test split
            X_train, X_test, y_train, y_test = train_test_split(X_group, y_group_classified, test_size=0.2, random_state=42)

            # Scale the training and test data
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)

            # Train only if there are multiple classes
            if len(y_train.unique()) > 1:
                model.fit(X_train_scaled, y_train)
                predictions = model.predict(X_test_scaled)

                # Store evaluation metrics
                accuracies.append(accuracy_score(y_test, predictions))
                precisions.append(precision_score(y_test, predictions, average='weighted', zero_division=0))
                recalls.append(recall_score(y_test, predictions, average='weighted', zero_division=0))

    # Compute the average metrics
    avg_metrics = {
        'accuracy': np.mean(accuracies) if accuracies else 0,
        'precision': np.mean(precisions) if precisions else 0,
        'recall': np.mean(recalls) if recalls else 0
    }

    return avg_metrics

In [65]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score

# Independent variables (predictors)
X_country = opera[['city population', 'country population', 'performances_season_by_country',
                   'perf_per_10k_ppl_co_pop', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Week_of_Year',
                   'Days_Since_Start', 'Month_1', 'Month_2', 'Month_3', 'Month_4',
                   'Month_5', 'Month_6', 'Month_7', 'Month_8', 'Month_9', 'Month_10',
                   'Month_11', 'Month_12', 'Weekday_0', 'Weekday_1', 'Weekday_2',
                   'Weekday_3', 'Weekday_4', 'Weekday_5', 'Weekday_6',
                   'Season Year_2013-01-01 00:00:00', 'Season Year_2014-01-01 00:00:00',
                   'Season Year_2015-01-01 00:00:00', 'Season Year_2016-01-01 00:00:00',
                   'Season Year_2017-01-01 00:00:00', 'Season Year_2018-01-01 00:00:00']]

# Dependent variable (growth at country level)
y_country = opera['country_change_from_previous_season']

# Convert continuous growth to categorical labels (Low, Medium, High)
q1 = opera['country_change_from_previous_season'].quantile(0.25)
q2 = opera['country_change_from_previous_season'].quantile(0.50)
q3 = opera['country_change_from_previous_season'].quantile(0.75)

bins = [-float('inf'), q1, q2, q3, float('inf')]
labels = ['Low', 'Medium', 'High', 'Very High']

y_country_classified = pd.cut(y_country, bins=bins, labels=labels)

# Train-test split
X_train, X_test, y_train_country, y_test_country = train_test_split(X_country, y_country_classified, test_size=0.2, random_state=42)

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test data (using the same scaler as the training data)
X_test_scaled = scaler.transform(X_test)

In [67]:
# Initialize LogisticRegression
lr_country = LogisticRegression(max_iter=10000, multi_class='ovr')

# Define the hyperparameter grid for tuning
param_grid = {
    'C': [0.1, 1, 10],  # Regularization strength
    'solver': ['liblinear', 'saga'],  # Use 'liblinear' and 'saga' or 'lbfgs' for L1
}

# Define the custom scoring function (e.g., F1 score)
custom_scorer = make_scorer(f1_score, average='weighted')

# GridSearchCV with custom scoring
grid_search = GridSearchCV(estimator=lr_country, param_grid=param_grid, scoring=custom_scorer, cv=5)
grid_search.fit(X_train_scaled, y_train_country)

# Output the best parameters and the best F1 score
print("Best Parameters:", grid_search.best_params_)
print("Best F1 Score:", grid_search.best_score_)

Best Parameters: {'C': 10, 'solver': 'saga'}
Best F1 Score: 0.628339186358349


In [71]:
# Best parameters found during the grid search
best_params = grid_search.best_params_

# Create the Logistic Regression model with the best parameters
lr_country = LogisticRegression(
    C=best_params['C'],
    solver=best_params['solver'],
    max_iter=10000,
    multi_class='ovr'  # Specify multiclass handling (One-vs-Rest)
)

# Initialize a container to store predictions with correct dtype (categorical or object)
predicted_growth_values = pd.Series(index=opera.index, dtype='object')

# Group the data by 'Country Name'
sub_region_groups = opera.groupby('sub-region')

# Loop through each country group to train and predict separately
for sub_region, group in sub_region_groups:
    # If the group has more than one sample and more than one class, split and train
    if len(group) > 1 and len(group['country_change_from_previous_season'].unique()) > 1:
        # Extract the features and target for this country group
        X_group = group[['city population', 'country population', 'performances_season_by_country',
                   'perf_per_10k_ppl_co_pop', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Week_of_Year',
                   'Days_Since_Start', 'Month_1', 'Month_2', 'Month_3', 'Month_4',
                   'Month_5', 'Month_6', 'Month_7', 'Month_8', 'Month_9', 'Month_10',
                   'Month_11', 'Month_12', 'Weekday_0', 'Weekday_1', 'Weekday_2',
                   'Weekday_3', 'Weekday_4', 'Weekday_5', 'Weekday_6',
                   'Season Year_2013-01-01 00:00:00', 'Season Year_2014-01-01 00:00:00',
                   'Season Year_2015-01-01 00:00:00', 'Season Year_2016-01-01 00:00:00',
                   'Season Year_2017-01-01 00:00:00', 'Season Year_2018-01-01 00:00:00']]
        y_group = group['country_change_from_previous_season']

        # Convert the target to categorical labels (same as before)
        y_group_classified = pd.cut(y_group, bins=bins, labels=labels)

        # Split data into train and test sets (80% train, 20% test)
        X_train, X_test, y_train_country, y_test_country = train_test_split(X_group, y_group_classified, test_size=0.2, random_state=42)

        # Scale the training data
        X_train_scaled = scaler.fit_transform(X_train)

        # **Only train if the target variable has more than one class**
        if len(y_train_country.unique()) > 1:
            # Train the model on the training data
            lr_country.fit(X_train_scaled, y_train_country)

            # Scale the test data using the same scaler
            X_test_scaled = scaler.transform(X_test)

            # Make predictions for the test set
            predictions = lr_country.predict(X_test_scaled)

            # Assign predictions to the correct indices in the original DataFrame
            predicted_growth_values.loc[X_test.index] = predictions  # Correctly assign predictions to test set rows
        else:
            print(f"Skipping the sub-region: {sub_region} due to insufficient class variation in the target.")
            continue
    else:
        # Skip the group if it has only one class or one sample
        print(f"Skipping the sub-region: {sub_region} due to insufficient class variation or sample size.")
        continue

# You can now inspect or save the results
opera['predicted_country_growth'] = predicted_growth_values

# Sort and get the top countries based on predicted growth
df_country_growth_sorted = opera[['Country Name', 'predicted_country_growth']].sort_values(by='predicted_country_growth', ascending=False)

# Drop duplicates and keep the row with the highest growth for each country
df_opera_country_growth_unique = df_country_growth_sorted.drop_duplicates(subset='Country Name', keep='first')

# Get top 10 countries with highest predicted growth
top_countries_predicted_lr1 = df_opera_country_growth_unique.head(10)
print(top_countries_predicted_lr1)

Skipping the sub-region: Northern Africa due to insufficient class variation in the target.
Skipping the sub-region: South-eastern Asia due to insufficient class variation in the target.
             Country Name predicted_country_growth
25243       United States                Very High
8480              Germany                Very High
8930                Spain                Very High
26500             Austria                Very High
11879      United Kingdom                Very High
10782  Russian Federation                Very High
11540             Ukraine                Very High
11283              Sweden                Very High
3136                Italy                Very High
29193             Hungary                Very High


In [73]:
lr_metrics_country1 = evaluate_model_performance(lr_country, X_country, y_country, sub_region_groups, scaler, bins, labels)

print(f"\n🔹 Average Logistic Regression Metrics Across Sub-Regions for Countries using GridSearch:")
for metric, value in lr_metrics_country1.items():
    print(f"   {metric}: {value:.4f}")


🔹 Average Logistic Regression Metrics Across Sub-Regions for Countries using GridSearch:
   accuracy: 0.9319
   precision: 0.9328
   recall: 0.9319


In [75]:
# Independent variables (predictors)
X_city = opera[['city population', 'country population', 'performances_season_by_country',
                   'perf_per_10k_ppl_co_pop', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Week_of_Year',
                   'Days_Since_Start', 'Month_1', 'Month_2', 'Month_3', 'Month_4',
                   'Month_5', 'Month_6', 'Month_7', 'Month_8', 'Month_9', 'Month_10',
                   'Month_11', 'Month_12', 'Weekday_0', 'Weekday_1', 'Weekday_2',
                   'Weekday_3', 'Weekday_4', 'Weekday_5', 'Weekday_6',
                   'Season Year_2013-01-01 00:00:00', 'Season Year_2014-01-01 00:00:00',
                   'Season Year_2015-01-01 00:00:00', 'Season Year_2016-01-01 00:00:00',
                   'Season Year_2017-01-01 00:00:00', 'Season Year_2018-01-01 00:00:00']]

# Dependent variable (growth at city level)
y_city = opera['city_change_from_previous_season']

# Convert continuous growth to categorical labels (Low, Medium, High)
q1 = opera['city_change_from_previous_season'].quantile(0.25)
q2 = opera['city_change_from_previous_season'].quantile(0.50)
q3 = opera['city_change_from_previous_season'].quantile(0.75)

bins = [-float('inf'), q1, q2, q3, float('inf')]
labels = ['Low', 'Medium', 'High', 'Very High']

y_city_classified = pd.cut(y_city, bins=bins, labels=labels)

# Train-test split
X_train, X_test, y_train_city, y_test_city = train_test_split(X_city, y_city_classified, test_size=0.2, random_state=42)

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test data (using the same scaler as the training data)
X_test_scaled = scaler.transform(X_test)

In [77]:
# Initialize LogisticRegression
lr_city = LogisticRegression(max_iter=10000, multi_class='ovr')

# Define the hyperparameter grid for tuning
param_grid = {
    'C': [0.1, 1, 10],  # Regularization strength
    'solver': ['liblinear', 'saga'],  # Use 'liblinear' and 'saga' or 'lbfgs' for L1
}

# Define the custom scoring function (e.g., F1 score)
custom_scorer = make_scorer(f1_score, average='weighted')

# GridSearchCV with custom scoring
grid_search = GridSearchCV(estimator=lr_city, param_grid=param_grid, scoring=custom_scorer, cv=5)
grid_search.fit(X_train_scaled, y_train_city)

# Output the best parameters and the best F1 score
print("Best Parameters:", grid_search.best_params_)
print("Best F1 Score:", grid_search.best_score_)

Best Parameters: {'C': 10, 'solver': 'saga'}
Best F1 Score: 0.3621151677341871


In [78]:
# Best parameters found during the grid search
best_params = grid_search.best_params_

# Create the Logistic Regression model with the best parameters
lr_city = LogisticRegression(
    C=best_params['C'],
    solver=best_params['solver'],
    max_iter=10000,
    multi_class='ovr'  # Specify multiclass handling (One-vs-Rest)
)

# Initialize a container to store predictions with correct dtype (categorical or object)
predicted_growth_values = pd.Series(index=opera.index, dtype='object')

# Still group the data by 'sub-region'
sub_region = opera.groupby('sub-region')

# Loop through each sub-region group to train and predict separately for cities
for sub_region, group in sub_region_groups:
    # If the group has more than one sample and more than one class, split and train
    if len(group) > 1 and len(group['city_change_from_previous_season'].unique()) > 1:
        # Extract the features and target for this sub-region group
        X_group = group[['city population', 'country population', 'performances_season_by_country',
                   'perf_per_10k_ppl_co_pop', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Week_of_Year',
                   'Days_Since_Start', 'Month_1', 'Month_2', 'Month_3', 'Month_4',
                   'Month_5', 'Month_6', 'Month_7', 'Month_8', 'Month_9', 'Month_10',
                   'Month_11', 'Month_12', 'Weekday_0', 'Weekday_1', 'Weekday_2',
                   'Weekday_3', 'Weekday_4', 'Weekday_5', 'Weekday_6',
                   'Season Year_2013-01-01 00:00:00', 'Season Year_2014-01-01 00:00:00',
                   'Season Year_2015-01-01 00:00:00', 'Season Year_2016-01-01 00:00:00',
                   'Season Year_2017-01-01 00:00:00', 'Season Year_2018-01-01 00:00:00']]
        y_group = group['city_change_from_previous_season']

        # Convert the target to categorical labels (same as before)
        y_group_classified = pd.cut(y_group, bins=bins, labels=labels)

        # Split data into train and test sets (80% train, 20% test)
        X_train, X_test, y_train_city, y_test_city = train_test_split(X_group, y_group_classified, test_size=0.2, random_state=42)

        # Scale the training data
        X_train_scaled = scaler.fit_transform(X_train)

        # **Only train if the target variable has more than one class**
        if len(y_train_city.unique()) > 1:
            # Train the model on the training data
            lr_city.fit(X_train_scaled, y_train_city)

            # Scale the test data using the same scaler
            X_test_scaled = scaler.transform(X_test)

            # Make predictions for the test set
            predictions = lr_city.predict(X_test_scaled)

            # Ensure the indices of predictions and X_test match
            predicted_growth_values.loc[X_test.index] = predictions  # Correctly assign predictions to test set rows
        else:
            print(f"Skipping sub-region: {sub_region} due to insufficient class variation in the target.")
            continue
    else:
        # Skip the group if it has only one class or one sample
        print(f"Skipping sub-region: {sub_region} due to insufficient class variation or sample size.")
        continue

# You can now inspect or save the results
opera['predicted_city_growth'] = predicted_growth_values

# Sort and get the top cities based on predicted growth
df_city_growth_sorted = opera[['iso', 'city', 'predicted_city_growth']].sort_values(by='predicted_city_growth', ascending=False)

# Drop duplicates and keep the row with the highest growth for each city
df_opera_city_growth_unique = df_city_growth_sorted.drop_duplicates(subset='city', keep='first')

# Get top 10 cities with highest predicted growth
top_cities_predicted_lr1 = df_opera_city_growth_unique.head(10)
print(top_cities_predicted_lr1)

      iso           city predicted_city_growth
25461  us        Houston             Very High
8679   de      Wiesbaden             Very High
17920  ru  St Petersburg             Very High
29224  hu           Pécs             Very High
33374  ca        Toronto             Very High
29228  hu         Szeged             Very High
8564   de     Regensburg             Very High
8783   dk      København             Very High
8819   ee        Tallinn             Very High
33381  ca      Vancouver             Very High


In [79]:
lr_metrics_city1 = evaluate_model_performance(lr_city, X_city, y_city, sub_region_groups, scaler, bins, labels)

print(f"\n🔹 Average Logistic Regression Metrics Across Sub-Regions for Cities using GridSearch:")
for metric, value in lr_metrics_city1.items():
    print(f"   {metric}: {value:.4f}")


🔹 Average Logistic Regression Metrics Across Sub-Regions for Cities using GridSearch:
   accuracy: 0.6248
   precision: 0.6075
   recall: 0.6248


Let's see if there's better results using RandomizedSearchCV instead of GridSearchCV

In [84]:
from sklearn.model_selection import RandomizedSearchCV

# Initialize LogisticRegression
lr_country = LogisticRegression(max_iter=20000, multi_class='ovr')

# Define the hyperparameter distribution for RandomizedSearchCV
param_dist = {
    'C': np.logspace(-3, 3, 7),  # Exponential range for C (e.g., 0.001, 0.01, 0.1, 1, 10, 100, 1000)
    'solver': ['liblinear', 'saga'],  # Solvers for logistic regression
    'max_iter': [1000, 10000, 20000],  # Number of iterations for convergence
}

# Define the custom scoring function (e.g., F1 score)
custom_scorer = make_scorer(f1_score, average='weighted')

# Update n_iter to match the number of combinations
random_search = RandomizedSearchCV(estimator=lr_country, 
                                   param_distributions=param_dist, 
                                   n_iter=100,  # Set n_iter to the number of parameter combinations
                                   cv=5, 
                                   n_jobs=-1, 
                                   verbose=2, 
                                   scoring=custom_scorer, 
                                   random_state=42)

# Fit the model with the scaled training data
random_search.fit(X_train_scaled, y_train_country)

# Output the best parameters and the best F1 score
print("Best Parameters:", random_search.best_params_)
print("Best F1 Score:", random_search.best_score_)

Fitting 5 folds for each of 42 candidates, totalling 210 fits


/opt/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 42 is smaller than n_iter=100. Running 42 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages

Best Parameters: {'solver': 'liblinear', 'max_iter': 1000, 'C': 1000.0}
Best F1 Score: 0.8633174024203953


In [88]:
# Best parameters found during the grid search
best_params = random_search.best_params_

# Create the Logistic Regression model with the best parameters
lr_country = LogisticRegression(
    C=best_params['C'],
    solver=best_params['solver'],
    max_iter=best_params['max_iter'],
    multi_class='ovr'  # Specify multiclass handling (One-vs-Rest)
)

# Initialize a container to store predictions with correct dtype (categorical or object)
predicted_growth_values = pd.Series(index=opera.index, dtype='object')

# Group the data by 'Country Name'
sub_region_groups = opera.groupby('sub-region')

# Loop through each country group to train and predict separately
for sub_region, group in sub_region_groups:
    # If the group has more than one sample and more than one class, split and train
    if len(group) > 1 and len(group['country_change_from_previous_season'].unique()) > 1:
        # Extract the features and target for this country group
        X_group = group[['city population', 'country population', 'performances_season_by_country',
                   'perf_per_10k_ppl_co_pop', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Week_of_Year',
                   'Days_Since_Start', 'Month_1', 'Month_2', 'Month_3', 'Month_4',
                   'Month_5', 'Month_6', 'Month_7', 'Month_8', 'Month_9', 'Month_10',
                   'Month_11', 'Month_12', 'Weekday_0', 'Weekday_1', 'Weekday_2',
                   'Weekday_3', 'Weekday_4', 'Weekday_5', 'Weekday_6',
                   'Season Year_2013-01-01 00:00:00', 'Season Year_2014-01-01 00:00:00',
                   'Season Year_2015-01-01 00:00:00', 'Season Year_2016-01-01 00:00:00',
                   'Season Year_2017-01-01 00:00:00', 'Season Year_2018-01-01 00:00:00']]
        y_group = group['country_change_from_previous_season']

        # Convert the target to categorical labels (same as before)
        y_group_classified = pd.cut(y_group, bins=bins, labels=labels)

        # Split data into train and test sets (80% train, 20% test)
        X_train, X_test, y_train_country, y_test_country = train_test_split(X_group, y_group_classified, test_size=0.2, random_state=42)

        # Scale the training data
        X_train_scaled = scaler.fit_transform(X_train)

        # **Only train if the target variable has more than one class**
        if len(y_train_country.unique()) > 1:
            # Train the model on the training data
            lr_country.fit(X_train_scaled, y_train_country)

            # Scale the test data using the same scaler
            X_test_scaled = scaler.transform(X_test)

            # Make predictions for the test set
            predictions = lr_country.predict(X_test_scaled)

            # Assign predictions to the correct indices in the original DataFrame
            predicted_growth_values.loc[X_test.index] = predictions  # Correctly assign predictions to test set rows
        else:
            print(f"Skipping country: {sub_region} due to insufficient class variation in the target.")
            continue
    else:
        # Skip the group if it has only one class or one sample
        print(f"Skipping country: {sub_region} due to insufficient class variation or sample size.")
        continue

# You can now inspect or save the results
opera['predicted_country_growth'] = predicted_growth_values

# Sort and get the top countries based on predicted growth
df_country_growth_sorted = opera[['Country Name', 'predicted_country_growth']].sort_values(by='predicted_country_growth', ascending=False)

# Drop duplicates and keep the row with the highest growth for each country
df_opera_country_growth_unique = df_country_growth_sorted.drop_duplicates(subset='Country Name', keep='first')

# Get top 10 countries with highest predicted growth
top_countries_predicted_lr2 = df_opera_country_growth_unique.head(10)
print(top_countries_predicted_lr2)

             Country Name predicted_country_growth
11328            Slovenia                Very High
9589              Hungary                Very High
9738                Italy                Very High
9621              Ireland                Very High
10022               Japan                Very High
10060  Korea, Republic of                Very High
10094           Lithuania                Very High
9279               France                Very High
9485              Croatia                Very High
9469               Greece                Very High


In [90]:
lr_metrics_country2 = evaluate_model_performance(lr_country, X_country, y_country, sub_region_groups, scaler, bins, labels)

print(f"\n🔹 Average Logistic Regression Metrics Across Sub-Regions for Countries using RandomizedSearch:")
for metric, value in lr_metrics_country2.items():
    print(f"   {metric}: {value:.4f}")


🔹 Average Logistic Regression Metrics Across Sub-Regions for Countries using RandomizedSearch:
   accuracy: 0.8424
   precision: 0.8310
   recall: 0.8424


In [92]:
# Initialize LogisticRegression
lr_city = LogisticRegression(max_iter=20000, multi_class='ovr')

# Define the hyperparameter distribution for RandomizedSearchCV
param_dist = {
    'C': np.logspace(-3, 3, 7),  # Exponential range for C (e.g., 0.001, 0.01, 0.1, 1, 10, 100, 1000)
    'solver': ['liblinear', 'saga'],  # Solvers for logistic regression
    'max_iter': [1000, 10000, 20000],  # Number of iterations for convergence
}

# Define the custom scoring function (e.g., F1 score)
custom_scorer = make_scorer(f1_score, average='weighted')

# RandomizedSearchCV with custom scoring
random_search = RandomizedSearchCV(estimator=lr_city, 
                                   param_distributions=param_dist, 
                                   n_iter=100,  # Number of random parameter combinations to try
                                   cv=5,  # Number of folds in cross-validation
                                   n_jobs=-1,  # Use all CPU cores
                                   verbose=2,  # Print progress messages
                                   scoring=custom_scorer,  # Use F1 score for evaluation
                                   random_state=42)  # For reproducibility

# Fit the model with the scaled training data
random_search.fit(X_train_scaled, y_train_city)

# Output the best parameters and the best F1 score
print("Best Parameters:", random_search.best_params_)
print("Best F1 Score:", random_search.best_score_)

Fitting 5 folds for each of 42 candidates, totalling 210 fits


/opt/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 42 is smaller than n_iter=100. Running 42 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages

Best Parameters: {'solver': 'saga', 'max_iter': 10000, 'C': 10.0}
Best F1 Score: 0.44952382430675286


In [96]:
# Best parameters found during the grid search
best_params = random_search.best_params_

# Initialize LogisticRegression with the best parameters
lr_city = LogisticRegression(
    C=best_params['C'],
    solver=best_params['solver'],
    max_iter=best_params['max_iter'],
    multi_class='ovr'  # Specify multiclass handling (One-vs-Rest)
)

# Initialize a container to store predictions with correct dtype (categorical or object)
predicted_growth_values = pd.Series(index=opera.index, dtype='object')

# Still group the data by 'sub-region'
sub_region_groups = opera.groupby('sub-region')

# Loop through each sub-region group to train and predict separately for cities
for sub_region, group in sub_region_groups:
    # If the group has more than one sample and more than one class, split and train
    if len(group) > 1 and len(group['city_change_from_previous_season'].unique()) > 1:
        # Extract the features and target for this sub-region group
        X_group = group[['city population', 'country population', 'performances_season_by_country',
                   'perf_per_10k_ppl_co_pop', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Week_of_Year',
                   'Days_Since_Start', 'Month_1', 'Month_2', 'Month_3', 'Month_4',
                   'Month_5', 'Month_6', 'Month_7', 'Month_8', 'Month_9', 'Month_10',
                   'Month_11', 'Month_12', 'Weekday_0', 'Weekday_1', 'Weekday_2',
                   'Weekday_3', 'Weekday_4', 'Weekday_5', 'Weekday_6',
                   'Season Year_2013-01-01 00:00:00', 'Season Year_2014-01-01 00:00:00',
                   'Season Year_2015-01-01 00:00:00', 'Season Year_2016-01-01 00:00:00',
                   'Season Year_2017-01-01 00:00:00', 'Season Year_2018-01-01 00:00:00']]
        y_group = group['city_change_from_previous_season']

        # Convert the target to categorical labels (same as before)
        y_group_classified = pd.cut(y_group, bins=bins, labels=labels)

        # Split data into train and test sets (80% train, 20% test)
        X_train, X_test, y_train_city, y_test_city = train_test_split(X_group, y_group_classified, test_size=0.2, random_state=42)

        # Scale the training data
        X_train_scaled = scaler.fit_transform(X_train)

        # **Only train if the target variable has more than one class**
        if len(y_train_city.unique()) > 1:
            # Train the model on the training data
            lr_city.fit(X_train_scaled, y_train_city)

            # Scale the test data using the same scaler
            X_test_scaled = scaler.transform(X_test)

            # Make predictions for the test set
            predictions = lr_city.predict(X_test_scaled)

            # Ensure the indices of predictions and X_test match
            predicted_growth_values.loc[X_test.index] = predictions  # Correctly assign predictions to test set rows
        else:
            print(f"Skipping sub-region: {city_name} due to insufficient class variation in the target.")
            continue
    else:
        # Skip the group if it has only one class or one sample
        print(f"Skipping sub-region: {city_name} due to insufficient class variation or sample size.")
        continue

# You can now inspect or save the results
opera['predicted_city_growth'] = predicted_growth_values

# Sort and get the top cities based on predicted growth
df_city_growth_sorted = opera[['iso', 'city', 'predicted_city_growth']].sort_values(by='predicted_city_growth', ascending=False)

# Drop duplicates and keep the row with the highest growth for each city
df_opera_city_growth_unique = df_city_growth_sorted.drop_duplicates(subset='city', keep='first')

# Get top 10 cities with highest predicted growth
top_cities_predicted_lr2 = df_opera_city_growth_unique.head(10)
print(top_cities_predicted_lr2)

      iso           city predicted_city_growth
25461  us        Houston             Very High
8679   de      Wiesbaden             Very High
17920  ru  St Petersburg             Very High
29224  hu           Pécs             Very High
33374  ca        Toronto             Very High
29228  hu         Szeged             Very High
8564   de     Regensburg             Very High
8783   dk      København             Very High
8819   ee        Tallinn             Very High
33381  ca      Vancouver             Very High


In [97]:
lr_metrics_city2 = evaluate_model_performance(lr_city, X_city, y_city, sub_region_groups, scaler, bins, labels)

print(f"\n🔹 Average Logistic Regression Metrics Across Sub-Regions for Cities using RandomizedSearch:")
for metric, value in lr_metrics_city2.items():
    print(f"   {metric}: {value:.4f}")


🔹 Average Logistic Regression Metrics Across Sub-Regions for Cities using RandomizedSearch:
   accuracy: 0.6248
   precision: 0.6075
   recall: 0.6248


Now I'm going to try Random Forest Classifier since the models for countries and cities were not very strong with logistic regression and might work better with RCF since it's a stronger model. I think a classifier will be better than a regression model for this since I categorized the target into "Very High", "High", "Medium" and "Low" making it more categorical than continuous.

In [100]:
from sklearn.ensemble import RandomForestClassifier

# Hyperparameter distributions for RandomizedSearchCV
param_dist = {
    'n_estimators': np.arange(100, 1001, 100),  # Randomly sample between 100 and 500 estimators
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],  # Can sample between 'sqrt' and 'log2'
    'bootstrap': [True, False]
}

# Initialize RandomForestClassifier
rf_country = RandomForestClassifier()

# RandomizedSearchCV with custom scoring (no n_iter)
random_search_rf = RandomizedSearchCV(estimator=rf_country, 
                                   param_distributions=param_dist,  # Param distributions for RandomizedSearchCV
                                   n_iter=100,  # Number of random combinations to sample
                                   cv=5, 
                                   n_jobs=-1, 
                                   verbose=2, 
                                   random_state=42)

# Fit the randomized search
random_search_rf.fit(X_train_scaled, y_train_country)

# Output the best parameters and the best F1 score
print("Best Parameters:", random_search_rf.best_params_)
print("Best F1 Score:", random_search_rf.best_score_)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best Parameters: {'n_estimators': 700, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': None, 'bootstrap': False}
Best F1 Score: 0.9997000499750126


In [101]:
# Best parameters found during the grid search
best_params = random_search_rf.best_params_

# Initialize RandomForestClassifier with the best parameters
rf_country = RandomForestClassifier(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    max_features=best_params['max_features'],
    bootstrap=best_params['bootstrap'],
    random_state=42
)

# Initialize a container to store predictions with correct dtype (categorical or object)
predicted_growth_values = pd.Series(index=opera.index, dtype='object')

# Still group the data by 'sub-region'
sub_region_groups = opera.groupby('sub-region')

# Initialize StandardScaler (to ensure consistent scaling)
scaler = StandardScaler()

# Loop through each country group to train and predict separately for cities
for sub_region, group in sub_region_groups:
    # If the group has more than one sample and more than one class, split and train
    if len(group) > 1 and len(group['country_change_from_previous_season'].unique()) > 1:
        # Extract the features and target for this country group
        X_group = group[['city population', 'country population', 'performances_season_by_country',
                   'perf_per_10k_ppl_co_pop', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Week_of_Year',
                   'Days_Since_Start', 'Month_1', 'Month_2', 'Month_3', 'Month_4',
                   'Month_5', 'Month_6', 'Month_7', 'Month_8', 'Month_9', 'Month_10',
                   'Month_11', 'Month_12', 'Weekday_0', 'Weekday_1', 'Weekday_2',
                   'Weekday_3', 'Weekday_4', 'Weekday_5', 'Weekday_6',
                   'Season Year_2013-01-01 00:00:00', 'Season Year_2014-01-01 00:00:00',
                   'Season Year_2015-01-01 00:00:00', 'Season Year_2016-01-01 00:00:00',
                   'Season Year_2017-01-01 00:00:00', 'Season Year_2018-01-01 00:00:00']]
        y_group = group['country_change_from_previous_season']

        # Convert the target to categorical labels (same as before)
        y_group_classified = pd.cut(y_group, bins=bins, labels=labels)

        # Split data into train and test sets (80% train, 20% test)
        X_train, X_test, y_train_country, y_test_country = train_test_split(X_group, y_group_classified, test_size=0.2, random_state=42)

        # Scale the training data
        X_train_scaled = scaler.fit_transform(X_train)

        # **Only train if the target variable has more than one class**
        if len(y_train_country.unique()) > 1:
            # Train the Random Forest Classifier with best_params
            rf_country.fit(X_train_scaled, y_train_country)  # ✅ Now fitting the model

            # Scale the test data using the same scaler
            X_test_scaled = scaler.transform(X_test)

            # Make predictions for the test set
            predictions = rf_country.predict(X_test_scaled)  # ✅ Using the fitted model

            # Assign predictions to the correct indices in the original DataFrame
            predicted_growth_values.loc[X_test.index] = predictions  # Correctly assign predictions to test set rows
        else:
            print(f"Skipping the sub-region: {sub_region} due to insufficient class variation in the target.")
            continue
    else:
        # Skip the group if it has only one class or one sample
        print(f"Skipping the sub-region: {sub_region} due to insufficient class variation or sample size.")
        continue

# You can now inspect or save the results
opera['predicted_country_growth'] = predicted_growth_values
df_country_growth_sorted = opera[['Country Name', 'predicted_country_growth']].sort_values(by='predicted_country_growth', ascending=False)

# Drop duplicates and keep the row with the highest growth for each country
df_opera_country_growth_unique = df_country_growth_sorted.drop_duplicates(subset='Country Name', keep='first')

# Get top 10 countries with highest predicted growth
top_countries_predicted_rf = df_opera_country_growth_unique.head(30)
print(top_countries_predicted_rf.head(10))

             Country Name predicted_country_growth
8237              Germany                Very High
10960  Russian Federation                Very High
11319            Slovenia                Very High
10407              Poland                Very High
10744              Serbia                Very High
18113              Sweden                Very High
12134       United States                Very High
11457             Ukraine                Very High
11617      United Kingdom                Very High
10258              Mexico                Very High


In [105]:
rf_metrics_country = evaluate_model_performance(rf_country, X_country, y_country, sub_region_groups, scaler, bins, labels)

print(f"\n🔹 Average Random Forest Metrics Across Sub-Regions for Countries")
for metric, value in rf_metrics_country.items():
    print(f"   {metric}: {value:.4f}")


🔹 Average Random Forest Metrics Across Sub-Regions for Countries
   accuracy: 0.9722
   precision: 0.9790
   recall: 0.9722


In [107]:
# Hyperparameter distributions for RandomizedSearchCV
param_dist = {
    'n_estimators': np.arange(100, 1001, 100),  # Randomly sample between 100 and 500 estimators
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],  # Can sample between 'sqrt' and 'log2'
    'bootstrap': [True, False]
}

# Initialize RandomForestClassifier
rf_city = RandomForestClassifier()

# RandomizedSearchCV with custom scoring (no n_iter)
random_search_rfy = RandomizedSearchCV(estimator=rf_city, 
                                   param_distributions=param_dist,  # Param distributions for RandomizedSearchCV
                                   n_iter=100,  # Number of random combinations to sample
                                   cv=5, 
                                   n_jobs=-1, 
                                   verbose=2, 
                                   random_state=42)

# Fit the randomized search
random_search_rfy.fit(X_train_scaled, y_train_city)

# Output the best parameters and the best F1 score
print("Best Parameters:", random_search_rfy.best_params_)
print("Best F1 Score:", random_search_rfy.best_score_)


Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] END ...........C=0.001, max_iter=1000, solver=liblinear; total time=   0.3s
[CV] END ................C=0.001, max_iter=1000, solver=saga; total time=   0.6s
[CV] END ..........C=0.001, max_iter=20000, solver=liblinear; total time=   0.2s
[CV] END ..........C=0.001, max_iter=20000, solver=liblinear; total time=   0.3s
[CV] END ............C=0.01, max_iter=1000, solver=liblinear; total time=   0.3s
[CV] END .................C=0.01, max_iter=1000, solver=saga; total time=   0.7s
[CV] END ................C=0.01, max_iter=10000, solver=saga; total time=   0.7s
[CV] END ...........C=0.01, max_iter=20000, solver=liblinear; total time=   0.5s
[CV] END .............C=0.1, max_iter=1000, solver=liblinear; total time=   0.9s
[CV] END ..................C=0.1, max_iter=1000, solver=saga; total time=   1.4s
[CV] END ............C=0.1, max_iter=20000, solver=liblinear; total time=   0.4s
[CV] END .................C=0.1, max_iter=2000

/opt/anaconda3/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best Parameters: {'n_estimators': 400, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 30, 'bootstrap': False}
Best F1 Score: 0.8860328335832083


In [110]:
# Best parameters found during the grid search
best_params = random_search_rfy.best_params_

# Initialize RandomForestClassifier with the best parameters
rf_city = RandomForestClassifier(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    max_features=best_params['max_features'],
    bootstrap=best_params['bootstrap'],
    random_state=42
)

# Initialize a container to store predictions with correct dtype (categorical or object)
predicted_growth_values = pd.Series(index=opera.index, dtype='object')

# Still group the data by 'sub-region'
sub_region_groups = opera.groupby('sub-region')

# Initialize StandardScaler (to ensure consistent scaling)
scaler = StandardScaler()

# Loop through each country group to train and predict separately for cities
for sub_region, group in sub_region_groups:
    # If the group has more than one sample and more than one class, split and train
    if len(group) > 1 and len(group['city_change_from_previous_season'].unique()) > 1:
        # Extract the features and target for this country group
        X_group = group[['city population', 'country population', 'performances_season_by_country',
                   'perf_per_10k_ppl_co_pop', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Week_of_Year',
                   'Days_Since_Start', 'Month_1', 'Month_2', 'Month_3', 'Month_4',
                   'Month_5', 'Month_6', 'Month_7', 'Month_8', 'Month_9', 'Month_10',
                   'Month_11', 'Month_12', 'Weekday_0', 'Weekday_1', 'Weekday_2',
                   'Weekday_3', 'Weekday_4', 'Weekday_5', 'Weekday_6',
                   'Season Year_2013-01-01 00:00:00', 'Season Year_2014-01-01 00:00:00',
                   'Season Year_2015-01-01 00:00:00', 'Season Year_2016-01-01 00:00:00',
                   'Season Year_2017-01-01 00:00:00', 'Season Year_2018-01-01 00:00:00']]
        y_group = group['city_change_from_previous_season']

        # Convert the target to categorical labels (same as before)
        y_group_classified = pd.cut(y_group, bins=bins, labels=labels)

        # Split data into train and test sets (80% train, 20% test)
        X_train, X_test, y_train_city, y_test_city = train_test_split(X_group, y_group_classified, test_size=0.2, random_state=42)

        # Scale the training data
        X_train_scaled = scaler.fit_transform(X_train)

        # **Only train if the target variable has more than one class**
        if len(y_train_city.unique()) > 1:
            # Train the Random Forest Classifier with best_params
            rf_city.fit(X_train_scaled, y_train_city)  # ✅ Now fitting the model

            # Scale the test data using the same scaler
            X_test_scaled = scaler.transform(X_test)

            # Make predictions for the test set
            predictions = rf_city.predict(X_test_scaled)  # ✅ Using the fitted model

            # Assign predictions to the correct indices in the original DataFrame
            predicted_growth_values.loc[X_test.index] = predictions  # Correctly assign predictions to test set rows
        else:
            print(f"Skipping the sub-region: {sub_region} due to insufficient class variation in the target.")
            continue
    else:
        # Skip the group if it has only one class or one sample
        print(f"Skipping the sub-region: {sub_region} due to insufficient class variation or sample size.")
        continue

# You can now inspect or save the results
opera['predicted_city_growth'] = predicted_growth_values
df_city_growth_sorted = opera[['iso', 'city', 'predicted_city_growth']].sort_values(by='predicted_city_growth', ascending=False)

# Drop duplicates and keep the row with the highest growth for each country
df_opera_city_growth_unique = df_city_growth_sorted.drop_duplicates(subset='city', keep='first')

# Get top 10 countries with highest predicted growth
top_cities_predicted_rf = df_opera_city_growth_unique.head(30)
print(top_cities_predicted_rf.head(10))

      iso        city predicted_city_growth
19184  us   Lancaster             Very High
25342  us      Denver             Very High
25331  us      Dayton             Very High
8256   de    Landshut             Very High
8300   de      Lubeck             Very High
8062   de       Halle             Very High
25427  us     Houston             Very High
8117   de    Hannover             Very High
8138   de  Heidelberg             Very High
8145   de  Heidenheim             Very High


In [112]:
rf_metrics_city = evaluate_model_performance(rf_city, X_city, y_city, sub_region_groups, scaler, bins, labels)

print(f"\n🔹 Average Random Forest Metrics Across Sub-Regions for Cities:")
for metric, value in rf_metrics_city.items():
    print(f"   {metric}: {value:.4f}")


🔹 Average Random Forest Metrics Across Sub-Regions for Cities:
   accuracy: 0.9387
   precision: 0.9375
   recall: 0.9387


Random Forest was much stronger at predicting with accuracy and precision. This is definitely a classifier problem rather than a regression model. I'm going to try one more model, the XGBoost.

In [115]:
!pip install xgboost

In [117]:
pip install --upgrade xgboost

Note: you may need to restart the kernel to use updated packages.


In [119]:
def evaluate_model_performance_1(model, X, y, sub_region_groups, scaler, bins, labels):
    """
    Trains and evaluates an XGBoost model for each sub-region, then returns the average accuracy, precision, and recall.

    Parameters:
    - model: The XGBoost model to train (e.g., XGBClassifier).
    - X: Feature DataFrame.
    - y: Target variable DataFrame.
    - sub_region_groups: Grouped data by sub-region.
    - scaler: StandardScaler instance.
    - bins: Bins for categorizing target values.
    - labels: Labels corresponding to bins.

    Returns:
    - Dictionary with average accuracy, precision, and recall.
    """
    accuracies, precisions, recalls = [], [], []

    for sub_region, group in sub_region_groups:
        if len(group) > 1 and len(group['country_change_from_previous_season'].unique()) > 1:
            # Extract features and target
            X_group = group[X.columns]
            y_group = group[y.name]

            # Convert target variable to categorical labels (Ensure labels start from 0)
            y_group_classified = pd.cut(y_group, bins=bins, labels=range(len(labels)))
            y_group_classified = y_group_classified.astype(int) - y_group_classified.min()

            # Train-test split
            X_train, X_test, y_train, y_test = train_test_split(X_group, y_group_classified, test_size=0.2, random_state=42)

            # Scale the training and test data
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)

            # Handle missing classes (Ensure XGBoost knows all possible labels)
            unique_classes = set(y_train.unique())
            all_classes = set(range(len(labels)))
            missing_classes = all_classes - unique_classes

            if missing_classes:
                for missing_class in missing_classes:
                    dummy_X = X_train_scaled.mean(axis=0) + np.random.normal(0, 0.01, X_train_scaled.shape[1])
                    dummy_X = dummy_X.reshape(1, -1)
                    dummy_y = np.array([missing_class])

                    X_train_scaled = np.vstack([X_train_scaled, dummy_X])
                    y_train = np.append(y_train, dummy_y)

            # Train only if there are multiple classes
            if len(np.unique(y_train)) > 1:
                model.fit(X_train_scaled, y_train)
                predictions = model.predict(X_test_scaled)

                # Store evaluation metrics
                accuracies.append(accuracy_score(y_test, predictions))
                precisions.append(precision_score(y_test, predictions, average='weighted', zero_division=0))
                recalls.append(recall_score(y_test, predictions, average='weighted', zero_division=0))

    # Compute the average metrics
    avg_metrics_1 = {
        'accuracy': np.mean(accuracies) if accuracies else 0,
        'precision': np.mean(precisions) if precisions else 0,
        'recall': np.mean(recalls) if recalls else 0
    }

    return avg_metrics_1

In [121]:
# Independent variables (predictors)

X_country = opera[['city population', 'country population', 'performances_season_by_country',
                   'perf_per_10k_ppl_co_pop', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Week_of_Year',
                   'Days_Since_Start', 'Month_1', 'Month_2', 'Month_3', 'Month_4',
                   'Month_5', 'Month_6', 'Month_7', 'Month_8', 'Month_9', 'Month_10',
                   'Month_11', 'Month_12', 'Weekday_0', 'Weekday_1', 'Weekday_2',
                   'Weekday_3', 'Weekday_4', 'Weekday_5', 'Weekday_6',
                   'Season Year_2013-01-01 00:00:00', 'Season Year_2014-01-01 00:00:00',
                   'Season Year_2015-01-01 00:00:00', 'Season Year_2016-01-01 00:00:00',
                   'Season Year_2017-01-01 00:00:00', 'Season Year_2018-01-01 00:00:00']]

# Dependent variable (growth at city level)
y_country = opera['country_change_from_previous_season']

# Compute quartiles for binning
q1 = y_country.quantile(0.25)
q2 = y_country.quantile(0.50)
q3 = y_country.quantile(0.75)

# Define bins and numeric labels
bins = [-float('inf'), q1, q2, q3, float('inf')]
labels = [0, 1, 2, 3]  # Low = 0, Medium = 1, High = 2, Very High = 3

# Convert to categorical numeric labels
y_country_classified = pd.cut(y_country, bins=bins, labels=labels).astype(int)

# Train-test split
X_train, X_test, y_train_country, y_test_country = train_test_split(X_country, y_country_classified, test_size=0.2, random_state=42)

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test data (using the same scaler as the training data)
X_test_scaled = scaler.transform(X_test)

In [123]:
import xgboost as xgb

# Define the hyperparameter search space
param_dist = {
    'n_estimators': np.arange(50, 500, 50),  # Number of trees
    'learning_rate': np.linspace(0.01, 0.3, 10),  # Learning rate
    'max_depth': np.arange(3, 10, 1),  # Tree depth
    'min_child_weight': np.arange(1, 10, 2),  # Minimum sum of instance weight needed in a child
    'gamma': np.linspace(0, 0.5, 5),  # Minimum loss reduction to make further partition
    'subsample': np.linspace(0.5, 1.0, 5),  # Fraction of samples used per tree
    'colsample_bytree': np.linspace(0.5, 1.0, 5),  # Fraction of features used per tree
    'reg_alpha': np.logspace(-3, 1, 5),  # L1 regularization
    'reg_lambda': np.logspace(-3, 1, 5),  # L2 regularization
}

# Initialize XGBoost classifier
xgb_country = xgb.XGBClassifier(objective='multi:softmax', num_class=3, random_state=42)

# Define a custom scoring function (F1-score)
custom_scorer = make_scorer(f1_score, average='weighted')

# Perform Randomized Search
random_search_xgb = RandomizedSearchCV(
    estimator=xgb_country,
    param_distributions=param_dist,
    n_iter=100,  # Number of random samples
    cv=5,  # 5-fold cross-validation
    scoring=custom_scorer,
    n_jobs=-1,  # Use all available processors
    verbose=2,
    random_state=42
)

# Fit the Randomized Search on training data (for country)
random_search_xgb.fit(X_train_scaled, y_train_country)

# Output the best parameters and best F1 score
print("Best Parameters:", random_search_xgb.best_params_)
print("Best F1 Score:", random_search_xgb.best_score_)


Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] END ...........C=0.001, max_iter=1000, solver=liblinear; total time=   0.3s
[CV] END ..........C=0.001, max_iter=10000, solver=liblinear; total time=   0.2s
[CV] END ..........C=0.001, max_iter=10000, solver=liblinear; total time=   0.3s
[CV] END ...............C=0.001, max_iter=10000, solver=saga; total time=   0.7s
[CV] END ............C=0.01, max_iter=1000, solver=liblinear; total time=   0.3s
[CV] END ............C=0.01, max_iter=1000, solver=liblinear; total time=   0.3s
[CV] END ...........C=0.01, max_iter=10000, solver=liblinear; total time=   0.7s
[CV] END ................C=0.01, max_iter=10000, solver=saga; total time=   0.7s
[CV] END ................C=0.01, max_iter=20000, solver=saga; total time=   0.8s
[CV] END ..................C=0.1, max_iter=1000, solver=saga; total time=   1.3s
[CV] END .................C=0.1, max_iter=10000, solver=saga; total time=   1.2s
[CV] END .................C=0.1, max_iter=2000

In [128]:
# Best parameters found during the grid search
best_params = random_search_xgb.best_params_

# Initialize XGBoost with the best parameters
xgb_country = xgb.XGBClassifier(
    n_estimators=best_params['n_estimators'],
    learning_rate=best_params['learning_rate'],
    max_depth=best_params['max_depth'],
    min_child_weight=best_params['min_child_weight'],
    gamma=best_params['gamma'],
    subsample=best_params['subsample'],
    colsample_bytree=best_params['colsample_bytree'],
    reg_alpha=best_params['reg_alpha'],
    reg_lambda=best_params['reg_lambda'],
    random_state=42
)

# Initialize a container to store predictions with correct dtype (categorical or object)
predicted_growth_values = pd.Series(index=opera.index, dtype='object')

# Still group the data by 'sub-region'
sub_region_groups = opera.groupby('sub-region')

# Initialize StandardScaler (to ensure consistent scaling)
scaler = StandardScaler()

# Loop through each sub-region to train and predict separately
for sub_region, group in sub_region_groups:
    # If the group has more than one sample and more than one class, split and train
    if len(group) > 1 and len(group['country_change_from_previous_season'].unique()) > 1:
        # Extract features and target
        X_group = group[['city population', 'country population', 'performances_season_by_country',
                   'perf_per_10k_ppl_co_pop', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Week_of_Year',
                   'Days_Since_Start', 'Month_1', 'Month_2', 'Month_3', 'Month_4',
                   'Month_5', 'Month_6', 'Month_7', 'Month_8', 'Month_9', 'Month_10',
                   'Month_11', 'Month_12', 'Weekday_0', 'Weekday_1', 'Weekday_2',
                   'Weekday_3', 'Weekday_4', 'Weekday_5', 'Weekday_6',
                   'Season Year_2013-01-01 00:00:00', 'Season Year_2014-01-01 00:00:00',
                   'Season Year_2015-01-01 00:00:00', 'Season Year_2016-01-01 00:00:00',
                   'Season Year_2017-01-01 00:00:00', 'Season Year_2018-01-01 00:00:00']]
        y_group = group['country_change_from_previous_season']

        # Convert the target to categorical labels (Ensuring labels start from 0)
        y_group_classified = pd.cut(y_group, bins=bins, labels=range(len(labels)))
        
        # Convert to integer type (XGBoost requires integer labels)
        y_group_classified = y_group_classified.astype(int)

        # Split into train-test sets (80-20 split)
        X_train, X_test, y_train_country, y_test_country = train_test_split(X_group, y_group_classified, test_size=0.2, random_state=42)

        # Scale the training data
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)  # Use same scaler for test data

        # Ensure all possible classes are included in training, even if missing
        if len(y_train_country.unique()) > 1:
            missing_classes = set(range(len(labels))) - set(y_train_country.unique())

            if missing_classes:
                for missing_class in missing_classes:
                    dummy_X = X_train_scaled.mean(axis=0) + np.random.normal(0, 0.01, X_train_scaled.shape[1])  # Add noise
                    dummy_X = dummy_X.reshape(1, -1)  
                    dummy_y = np.array([missing_class])

                    X_train_scaled = np.vstack([X_train_scaled, dummy_X])
                    y_train_country = np.append(y_train_country, dummy_y)

            # Train the model
            xgb_country.fit(X_train_scaled, y_train_country)

            # Make predictions
            predictions = xgb_country.predict(X_test_scaled)

            # Assign predictions to correct indices in the original DataFrame
            predicted_growth_values.loc[X_test.index] = predictions
        else:
            print(f"Skipping {sub_region} due to insufficient class variation.")
            continue
    else:
        print(f"Skipping {sub_region} due to insufficient data.")
        continue

# Fill NaN values in case of skipped regions
opera['predicted_country_growth'] = predicted_growth_values.fillna("No Prediction")

# Convert predictions to numeric where possible, keeping "No Prediction" as-is
opera['numeric_growth'] = pd.to_numeric(opera['predicted_country_growth'], errors='coerce')

# Sort using numeric values (NaNs get pushed to the bottom)
df_country_growth_sorted = opera[['Country Name', 'predicted_country_growth', 'numeric_growth']].sort_values(by='numeric_growth', ascending=False)

# Drop temp numeric column after sorting
df_country_growth_sorted = df_country_growth_sorted.drop(columns='numeric_growth')

# Drop duplicates and keep the row with the highest growth for each country
df_opera_country_growth_unique = df_country_growth_sorted.drop_duplicates(subset='Country Name', keep='first')

# Get top 10 countries with highest predicted growth
top_countries_predicted_xgb = df_opera_country_growth_unique.head(30)
print(top_countries_predicted_xgb.head(10))

             Country Name predicted_country_growth
8303              Germany                        3
9937                Italy                        3
9984                Japan                        3
10060  Korea, Republic of                        3
25609       United States                        3
11739      United Kingdom                        3
10407              Poland                        3
10159              Latvia                        3
10256              Mexico                        3
9536              Hungary                        3


In [130]:
xgb_metrics_country = evaluate_model_performance_1(xgb_country, X_country, y_country, sub_region_groups, scaler, bins, labels)

print(f"\n🔹 Average XGBoost Metrics Across Sub-Regions for Countries")
for metric, value in xgb_metrics_country.items():
    print(f"   {metric}: {value:.4f}")


🔹 Average XGBoost Metrics Across Sub-Regions for Countries
   accuracy: 0.9089
   precision: 0.8800
   recall: 0.9089


In [132]:
# Independent variables (predictors)
X_city = opera[['city population', 'country population', 'performances_season_by_country',
                   'perf_per_10k_ppl_co_pop', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Week_of_Year',
                   'Days_Since_Start', 'Month_1', 'Month_2', 'Month_3', 'Month_4',
                   'Month_5', 'Month_6', 'Month_7', 'Month_8', 'Month_9', 'Month_10',
                   'Month_11', 'Month_12', 'Weekday_0', 'Weekday_1', 'Weekday_2',
                   'Weekday_3', 'Weekday_4', 'Weekday_5', 'Weekday_6',
                   'Season Year_2013-01-01 00:00:00', 'Season Year_2014-01-01 00:00:00',
                   'Season Year_2015-01-01 00:00:00', 'Season Year_2016-01-01 00:00:00',
                   'Season Year_2017-01-01 00:00:00', 'Season Year_2018-01-01 00:00:00']]

# Dependent variable (growth at city level)
y_city = opera['city_change_from_previous_season']

# Compute quartiles for binning
q1 = y_city.quantile(0.25)
q2 = y_city.quantile(0.50)
q3 = y_city.quantile(0.75)

# Define bins and numeric labels
bins = [-float('inf'), q1, q2, q3, float('inf')]
labels = [0, 1, 2, 3]  # Low = 0, Medium = 1, High = 2, Very High = 3

# Convert to categorical numeric labels
y_city_classified = pd.cut(y_city, bins=bins, labels=labels).astype(int)

# Train-test split
X_train, X_test, y_train_city, y_test_city = train_test_split(X_city, y_city_classified, test_size=0.2, random_state=42)

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test data (using the same scaler as the training data)
X_test_scaled = scaler.transform(X_test)

In [134]:
# Define the hyperparameter search space
param_dist = {
    'n_estimators': np.arange(50, 500, 50),  # Number of trees
    'learning_rate': np.linspace(0.01, 0.3, 10),  # Learning rate
    'max_depth': np.arange(3, 10, 1),  # Tree depth
    'min_child_weight': np.arange(1, 10, 2),  # Minimum sum of instance weight needed in a child
    'gamma': np.linspace(0, 0.5, 5),  # Minimum loss reduction to make further partition
    'subsample': np.linspace(0.5, 1.0, 5),  # Fraction of samples used per tree
    'colsample_bytree': np.linspace(0.5, 1.0, 5),  # Fraction of features used per tree
    'reg_alpha': np.logspace(-3, 1, 5),  # L1 regularization
    'reg_lambda': np.logspace(-3, 1, 5),  # L2 regularization
}

# Initialize XGBoost classifier
xgb_city = xgb.XGBClassifier(objective='multi:softmax', num_class=3, random_state=42)

# Define a custom scoring function (F1-score)
custom_scorer = make_scorer(f1_score, average='weighted')

# Perform Randomized Search
random_search_xgby = RandomizedSearchCV(
    estimator=xgb_city,
    param_distributions=param_dist,
    n_iter=100,  # Number of random samples
    cv=5,  # 5-fold cross-validation
    scoring=custom_scorer,
    n_jobs=-1,  # Use all available processors
    verbose=2,
    random_state=42
)

# Fit the Randomized Search on training data (for country)
random_search_xgby.fit(X_train_scaled, y_train_city)

# Output the best parameters and best F1 score
print("Best Parameters:", random_search_xgby.best_params_)
print("Best F1 Score:", random_search_xgby.best_score_)


Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] END bootstrap=True, max_depth=20, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=600; total time=   8.2s
[CV] END bootstrap=True, max_depth=10, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.1s
[CV] END bootstrap=False, max_depth=None, max_features=log2, min_samples_leaf=4, min_samples_split=10, n_estimators=500; total time=   9.5s
[CV] END bootstrap=False, max_depth=20, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=400; total time=   8.3s
[CV] END bootstrap=True, max_depth=30, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=700; total time=  13.5s
[CV] END bootstrap=False, max_depth=None, max_features=log2, min_samples_leaf=2, min_samples_split=2, n_estimators=800; total time=  17.9s
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimator

In [138]:
# Best parameters found during the grid search
best_params = random_search_xgby.best_params_

# Initialize XGBoost with the best parameters
xgb_city = xgb.XGBClassifier(
    n_estimators=best_params['n_estimators'],
    learning_rate=best_params['learning_rate'],
    max_depth=best_params['max_depth'],
    min_child_weight=best_params['min_child_weight'],
    gamma=best_params['gamma'],
    subsample=best_params['subsample'],
    colsample_bytree=best_params['colsample_bytree'],
    reg_alpha=best_params['reg_alpha'],
    reg_lambda=best_params['reg_lambda'],
    random_state=42
)

# Initialize a container to store predictions with correct dtype (categorical or object)
predicted_growth_values = pd.Series(index=opera.index, dtype='object')

# Still group the data by 'sub-region'
sub_region_groups = opera.groupby('sub-region')

# Initialize StandardScaler (to ensure consistent scaling)
scaler = StandardScaler()

# Loop through each sub-region to train and predict separately
for sub_region, group in sub_region_groups:
    # If the group has more than one sample and more than one class, split and train
    if len(group) > 1 and len(group['city_change_from_previous_season'].unique()) > 1:
        # Extract features and target
        X_group = group[['city population', 'country population', 'performances_season_by_country',
                   'perf_per_10k_ppl_co_pop', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Week_of_Year',
                   'Days_Since_Start', 'Month_1', 'Month_2', 'Month_3', 'Month_4',
                   'Month_5', 'Month_6', 'Month_7', 'Month_8', 'Month_9', 'Month_10',
                   'Month_11', 'Month_12', 'Weekday_0', 'Weekday_1', 'Weekday_2',
                   'Weekday_3', 'Weekday_4', 'Weekday_5', 'Weekday_6',
                   'Season Year_2013-01-01 00:00:00', 'Season Year_2014-01-01 00:00:00',
                   'Season Year_2015-01-01 00:00:00', 'Season Year_2016-01-01 00:00:00',
                   'Season Year_2017-01-01 00:00:00', 'Season Year_2018-01-01 00:00:00']]
        y_group = group['city_change_from_previous_season']

        # Convert the target to categorical labels (Ensuring labels start from 0)
        y_group_classified = pd.cut(y_group, bins=bins, labels=range(len(labels)))
        
        # Convert to integer type (XGBoost requires integer labels)
        y_group_classified = y_group_classified.astype(int)

        # Split into train-test sets (80-20 split)
        X_train, X_test, y_train_city, y_test_city = train_test_split(X_group, y_group_classified, test_size=0.2, random_state=42)

        # Scale the training data
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)  # Use same scaler for test data

        # Ensure all possible classes are included in training, even if missing
        if len(y_train_city.unique()) > 1:
            missing_classes = set(range(len(labels))) - set(y_train_city.unique())

            if missing_classes:
                for missing_class in missing_classes:
                    dummy_X = X_train_scaled.mean(axis=0) + np.random.normal(0, 0.01, X_train_scaled.shape[1])  # Add noise
                    dummy_X = dummy_X.reshape(1, -1)  
                    dummy_y = np.array([missing_class])

                    X_train_scaled = np.vstack([X_train_scaled, dummy_X])
                    y_train_city = np.append(y_train_city, dummy_y)

            # Train the model
            xgb_city.fit(X_train_scaled, y_train_city)

            # Make predictions
            predictions = xgb_city.predict(X_test_scaled)

            # Assign predictions to correct indices in the original DataFrame
            predicted_growth_values.loc[X_test.index] = predictions
        else:
            print(f"Skipping {sub_region} due to insufficient class variation.")
            continue
    else:
        print(f"Skipping {sub_region} due to insufficient data.")
        continue

# Fill NaN values in case of skipped regions
opera['predicted_city_growth'] = predicted_growth_values.fillna("No Prediction")

# Convert predictions to numeric where possible, keeping "No Prediction" as-is
opera['numeric_growth'] = pd.to_numeric(opera['predicted_city_growth'], errors='coerce')

# Sort using numeric values (NaNs get pushed to the bottom)
df_city_growth_sorted = opera[['iso', 'city', 'predicted_city_growth', 'numeric_growth']].sort_values(by='numeric_growth', ascending=False)

# Drop temp numeric column after sorting
df_city_growth_sorted = df_city_growth_sorted.drop(columns='numeric_growth')

# Drop duplicates and keep the row with the highest growth for each country
df_opera_city_growth_unique = df_city_growth_sorted.drop_duplicates(subset='city', keep='first')

# Get top 10 countries with highest predicted growth
top_cities_predicted_xgb = df_opera_city_growth_unique.head(30)
print(top_cities_predicted_xgb.head(10))

      iso          city predicted_city_growth
19184  us     Lancaster                     3
8016   de    Greifswald                     3
24026  ru        Moscow                     3
7984   de          Gera                     3
8009   de      Goerlitz                     3
24425  se     Stockholm                     3
7914   de     Flensburg                     3
23865  rs       Beograd                     3
23952  ru   Krasnoyarsk                     3
23922  ru  Ekaterinburg                     3


In [140]:
xgb_metrics_city = evaluate_model_performance_1(xgb_city, X_city, y_city, sub_region_groups, scaler, bins, labels)

print(f"\n🔹 Average XGBoost Metrics Across Sub-Regions for Countries")
for metric, value in xgb_metrics_city.items():
    print(f"   {metric}: {value:.4f}")


🔹 Average XGBoost Metrics Across Sub-Regions for Countries
   accuracy: 0.9290
   precision: 0.9213
   recall: 0.9290


In [144]:
# These variables are already defined in your environment
metric_vars = [
    "lr_metrics_country1", "lr_metrics_country2", "rf_metrics_country", "xgb_metrics_country", 
    "lr_metrics_city1", "lr_metrics_city2", "rf_metrics_city", "xgb_metrics_city"
]

# Create a dictionary to store the metric values
metrics_data = {}

# Iterate through each metric variable and retrieve its values dynamically
for var in metric_vars:
    if var in globals():  # Check if the variable exists in the global scope
        metrics_data[var] = globals()[var]  # Retrieve its value

# Convert to DataFrame with NaN where values are missing
final_df = pd.DataFrame(metrics_data).T

# Reorder columns for clarity
final_df = final_df[[ "accuracy", "precision", "recall"]]

# Display the final structured DataFrame
print(final_df)

                     accuracy  precision    recall
lr_metrics_country1  0.931912   0.932753  0.931912
lr_metrics_country2  0.842413   0.830957  0.842413
rf_metrics_country   0.972163   0.979044  0.972163
xgb_metrics_country  0.908950   0.879975  0.908950
lr_metrics_city1     0.624821   0.607470  0.624821
lr_metrics_city2     0.624821   0.607470  0.624821
rf_metrics_city      0.938745   0.937542  0.938745
xgb_metrics_city     0.928977   0.921337  0.928977


In [205]:
# Step 1: Add 'Model' column
top_countries_predicted_rf = top_countries_predicted_rf.copy()
top_countries_predicted_rf['Model'] = 'RF - Country'

top_countries_predicted_xgb = top_countries_predicted_xgb.copy()
top_countries_predicted_xgb['Model'] = 'XGB - Country'

# Step 2: Ensure no duplicates before merging and take more countries from each model
top_countries_predicted_rf = top_countries_predicted_rf.head(50).drop_duplicates(subset=['Country Name'])
top_countries_predicted_xgb = top_countries_predicted_xgb.head(50).drop_duplicates(subset=['Country Name'])

# Step 3: Merge RF and XGB predictions based on 'Country Name'
merged = pd.merge(
    top_countries_predicted_rf, 
    top_countries_predicted_xgb, 
    on=['Country Name'], 
    how='outer', 
    suffixes=('_rf', '_xgb')
)

# Step 4: Convert growth values to numeric to avoid TypeErrors
merged['predicted_country_growth_rf'] = pd.to_numeric(merged['predicted_country_growth_rf'], errors='coerce')
merged['predicted_country_growth_xgb'] = pd.to_numeric(merged['predicted_country_growth_xgb'], errors='coerce')

# Step 5: Determine the correct 'Model' label
merged['Model'] = merged.apply(
    lambda row: 'Both' if not pd.isna(row['Model_rf']) and not pd.isna(row['Model_xgb'])
    else row['Model_rf'] if not pd.isna(row['Model_rf']) 
    else row['Model_xgb'], 
    axis=1
)

# Step 6: Use the highest predicted_country_growth from RF or XGB
merged['predicted_country_growth'] = merged[['predicted_country_growth_rf', 'predicted_country_growth_xgb']].max(axis=1)

# Step 7: Select only necessary columns
top_countries_merged = merged[['Country Name', 'predicted_country_growth', 'Model']]

# Step 8: Fill missing growth values (optional, depending on data)
top_countries_merged = top_countries_merged.copy()
top_countries_merged.loc[:, 'predicted_country_growth'] = top_countries_merged['predicted_country_growth'].fillna(0)

# Step 9: Sort by predicted growth (descending) and then by country name
top_countries_merged = top_countries_merged.sort_values(by=['predicted_country_growth', 'Country Name'], ascending=[False, True])

# Step 10: Select top 20 unique countries
top_30_countries = top_countries_merged.head(30)

# Display results
print("Top 30 Countries Predictions (Grouped by Country):")
print(top_30_countries)

Top 30 Countries Predictions (Grouped by Country):
          Country Name  predicted_country_growth          Model
26           Argentina                       3.0           Both
35           Australia                       3.0  XGB - Country
28             Belgium                       3.0           Both
34               China                       3.0  XGB - Country
30      Czech Republic                       3.0  XGB - Country
10             Denmark                       3.0           Both
11             Estonia                       3.0           Both
13              France                       3.0           Both
0              Germany                       3.0           Both
19             Hungary                       3.0           Both
31             Ireland                       3.0  XGB - Country
15               Italy                       3.0           Both
17               Japan                       3.0           Both
18  Korea, Republic of                       3.0     

In [207]:
common_countries = pd.merge(
    top_countries_merged[['Country Name']],  # Only keep relevant columns
    top_countries[['Country Name']],     # Compare against full dataset
    on=['Country Name'],              # Match on both country 
    how='inner'                      # Keep only common pairs
)

print("Countries that exist in both datasets:")
print(common_countries)
print(common_countries.shape)

Countries that exist in both datasets:
          Country Name
0            Argentina
1            Australia
2                China
3              Estonia
4              Ireland
5                Italy
6   Korea, Republic of
7               Latvia
8          Netherlands
9          New Zealand
10                Peru
11              Poland
12  Russian Federation
13            Slovenia
14              Sweden
15          Kazakhstan
16             Uruguay
17          Uzbekistan
(18, 1)


After running multiple predictive models, the findings on the best emerging markets for opera show a strong correlation between predictive modeling results and visual data exploration. Among the top countries merged (compiled from the best-performing country models), 18 countries overlap, reinforcing their potential as emerging markets. A strategic approach for a traveling opera company could involve a balanced model: using well-established opera hubs as anchor locations during their off-seasons—such as Florence, and other cities in Italy, and various Austrian cities, where seasonality trends differ from other European markets. This company could then take the same productions to emerging markets during peak seasons in their anchor cities, tapping into new audiences in places like Sweden, the Netherlands, Poland, Hungary, and Uzbekistan—markets that are slowly expanding and present lower stakes for early investment.